This notebook uses a naive bayes classifier and a SVM to classify messages as spam or not spam.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
data = pd.read_csv("../input/spam.csv",encoding='latin-1')
data.loc[(data['v1'] == 'ham'), 'v1'] = 0
data.loc[(data['v1'] == 'spam'), 'v1'] = 1
data.head()

As the preview of the data above shows there are three useless columns, these should be removed. I will also rename the remaining columns as "v1" and "v2" are not descriptive.

In [ ]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

In [ ]:
data['length'] = data['text'].apply(len)
data.head()

In order to apply a model, the necessary preprocessing must be completed. For text classification, usual preprocessing includes removing stop words (words that don't provide useful meaning, i.e. "and" "or"). Also the characters are converted to a single case (the below function converts to lower case). The function below then stems each word (this means that it replaces a word with the root of that word, for example "tasted" or "tasting" would become "taste").

(Below preprocessing function from Evgeny Volkov)

In [ ]:
def pre_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

The below code copies the text data column, so any processing is not compelted on the original data. And then uses a TFIDF vectoriser to provide useful numerical values related to the data. TFIDF (term frequency - inverse document frequency) is a statistical method to tell how important a word is to a particular document by increasing the numerical value for an occurrence in the specific document but decreasing relative to number of occurrences in the entire corpus. 

After this, a function available in the sklearn library is used to randomly assign training and test data to train and test the machine learning models. 

In [ ]:
textFeatures = data['text'].copy()
textFeatures = textFeatures.apply(pre_process)
vectorizer = TfidfVectorizer("english")
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class'], test_size=0.3, random_state=111)
labels_train_binary = np.array(labels_train, dtype=np.uint8)
labels_test_binary = np.array(labels_test, dtype=np.uint8)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train_binary)
prediction = svc.predict(features_test)
f1_score(labels_test_binary, prediction)

The following code trains and test a Multinomial Naive Bayes Model using sklearn. It can be seen that is provided a slightly more accurate result than the SVM model so should therefore be used. There are many other models that may be more suitable for this dataset, however both of these model produce sufficient results. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train_binary)
prediction = mnb.predict(features_test)
f1_score(labels_test_binary, prediction)